In [0]:
import requests
import pandas as pd

# 🔑 Replace with your NewsAPI key
API_KEY = os.getenv("NEWS_API_KEY")

# Example: Get top US headlines
url = f"https://newsapi.org/v2/top-headlines?country=us&apiKey={API_KEY}"

response = requests.get(url).json()

# Extract relevant fields
articles = response.get("articles", [])
df = pd.DataFrame([{
    "title": a.get("title"),
    "author": a.get("author"),
    "publishedAt": a.get("publishedAt"),
    "content": a.get("content"),
    "url": a.get("url")
} for a in articles])

print(df.head())


                                               title  ...                                                url
0  Multiple people shot at New Hampshire country ...  ...  https://www.cnn.com/2025/09/20/us/nashua-new-h...
1  Democratic leaders in Congress demand a meetin...  ...  https://apnews.com/article/congress-government...
2  New H-1B visa fee will not apply to existing h...  ...  https://www.axios.com/2025/09/20/trump-h-1b-im...
3  Angeli, Syracuse upset Clemson 34-21; Dabo Swi...  ...  https://apnews.com/article/syracuse-clemson-sc...
4  Man arrested after drive-by shooting at ABC TV...  ...  https://www.npr.org/2025/09/20/nx-s1-5548558/a...

[5 rows x 5 columns]


In [0]:
CATEGORIES = [
    "business",
    "entertainment",
    "general",
    "health",
    "science",
    "sports",
    "technology"
]

In [0]:
def fetch_news(category=None, limit=10):
    if category:  # If user picks a category
        url = f"https://newsapi.org/v2/top-headlines?country=us&category={category}&apiKey={API_KEY}"
    else:  # Default = all categories
        url = f"https://newsapi.org/v2/top-headlines?country=us&apiKey={API_KEY}"
    
    response = requests.get(url).json()
    articles = response.get("articles", [])[:limit]
    
    data = []
    for a in articles:
        text = (a.get("description") or "") + " " + (a.get("content") or "")
        data.append({
            "title": a.get("title"),
            "content": text,
            "url": a.get("url"),
            "publishedAt": a.get("publishedAt")
        })
    return pd.DataFrame(data)

In [0]:
pip install transformers


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 MB 166.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 140.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 19.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
%restart_python

In [0]:
# Load summarizer model (make sure torch is installed!)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [0]:
def summarize_article(text):
    if not text or len(text.split()) < 20:
        return "Not enough text to summarize."
    
    input_len = len(text.split())
    max_len = min(120, int(input_len * 0.6))   # ~60% of input
    min_len = max(30, int(input_len * 0.3))    # ~30% of input
    
    summary = summarizer(
        text,
        max_length=max_len,
        min_length=min_len,
        do_sample=False
    )
    return summary[0]['summary_text']

In [0]:
# Step 3: Run with User Choice
# -------------------------------
# Example: user picks category
user_choice = "business"   # can be any from CATEGORIES, or None for all
limit = 5

df = fetch_news(user_choice if user_choice != "All" else None, limit=limit)
df["summary"] = df["content"].apply(summarize_article)

# Display results
for i, row in df.iterrows():
    print(f"\n {row['title']}")
    print(f" Published: {row['publishedAt']}")
    print(f" {row['url']}")
    print(f" Summary: {row['summary']}")
    print("-" * 80)

Your min_length=30 must be inferior than your max_length=27.
/local_disk0/.ephemeral_nfs/envs/pythonEnv-55c792e3-b43c-486c-bfce-bf0f0ac05b82/lib/python3.12/site-packages/transformers/generation/utils.py:1618: UserWarning: Unfeasible length constraints: `min_length` (30) is larger than the maximum possible length (27). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
Your min_length=30 must be inferior than your max_length=25.
/local_disk0/.ephemeral_nfs/envs/pythonEnv-55c792e3-b43c-486c-bfce-bf0f0ac05b82/lib/python3.12/site-packages/transformers/generation/utils.py:1618: UserWarning: Unfeasible length constraints: `min_length` (30) is larger than the maximum possible length (25). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(



 Here's How Much A 3-Year-Old Ford Bronco Has Depreciated So Far - Jalopnik
 Published: 2025-09-20T22:05:00Z
 https://www.jalopnik.com/1968781/ford-bronco-depreciation-3-year/
 Summary: After three years, your Ford Bronco's value has likely dropped by nearly 36%. Some cars depreciate much faster than others for a multitude of reasons. If a large number of
--------------------------------------------------------------------------------

 FAA says ‘multiple failures’ in technology caused DFW, Love Field equipment outage - Dallas News
 Published: 2025-09-20T20:48:45Z
 https://www.dallasnews.com/business/airlines/2025/09/20/faa-says-multiple-failures-in-technology-caused-dfw-love-field-equipment-outage/
 Summary: Both airports have returned to normal operations, FAA says. Ground stop and equipment outage caused hundreds of flights to be canceled.
--------------------------------------------------------------------------------

 Bitcoin defies September slump: Mapping BTC’s path to $130K -